In [1]:
import os
import pandas as pd
import json
import numpy as np
from ipyleaflet import Map, GeoJSON, LayersControl, LayerGroup, basemaps, basemap_to_tiles
import copy
import sys
import ipyleaflet

In [2]:
def parse_src(name):
    if "国产" in name:
        return "国产"
    elif "进口" in name:
        return "进口"
    else:
        return np.nan
    
def include_name(x):
    if not isnan_wrapper(x):
        return "长春生物" in x
    return False

def isnan_wrapper(x):
    try:
        return np.isnan(x)
    except:
        return False

# 数据预处理：
* 浙江省的数据格式和别的省份不一样
* 移除疫苗名字前后的空格

In [3]:
BASE_DIR = '#YOUR_OWN_PATH'
csv_files = os.listdir(BASE_DIR) 
dfs = [pd.read_csv(os.path.join(BASE_DIR, csv_file), engine='python', encoding='utf-8') for csv_file in csv_files]

In [4]:
dfs[11]['src'] = map(parse_src, dfs[11]['品名'])
dfs[11].rename({"供应商/生产企业" : "create_company",
                "中标价（元）": "price",
                "品名" : "name"
               }, axis = 1, inplace = True)
dfs[11]['prov'] = csv_files[11].strip(".csv")
dfs[11] = dfs[11][['name', 'src', 'create_company', 'prov', 'price']].copy()

In [5]:
final_df = pd.concat(dfs, sort=True)
final_df.name = final_df.name.apply(lambda x: x.strip())

has_data_provs = final_df.prov.unique()
vaccinations_affected = final_df[final_df.create_company.apply(include_name)].name.unique()

# GeoJson 可视化

In [6]:
BASE_DIR2 = '#YOUR_OWN_PATH'
with open(os.path.join(BASE_DIR2,  'zh-mainland-provinces.geojson'), 'r', encoding='utf-8') as f:
    geo_data = json.load(f)
    
for feature in geo_data['features']:
    if feature['properties']['name_local'] is None:
        feature['properties']['name_local'] = "青海|青海"

In [7]:
def prov_affected(prov, included_prov):
    # return True if prov is in included_prov
    for a_prov in included_prov:
        if prov in a_prov:
            return True
    return False

def prov_included(prov, has_data_provs):
    # return True if prov is in has_data_provs
    for a_prov in has_data_provs:
        if prov in a_prov:
            return True
    return False

def prepare_geo_json(vac_name, final_df, geo_data, has_data_provs):
    part_df = final_df[np.logical_and(final_df.name == vac_name, '长春生物' == final_df.create_company)]
    included_prov = part_df.prov.unique()
    geo_data_copy = copy.deepcopy(geo_data)
    for idx, feature in enumerate(geo_data_copy['features']):
        name = feature['properties']['name_local']
        if not name:
            continue
        # 黑龙江繁体在前 - hardcode
        if idx != 30:
            prov = name.split("|")[-1].encode('utf-8')
        else:
            prov = name.split("|")[0].encode('utf-8')

        if not prov_included(prov.decode('utf-8'), has_data_provs): # no data - grey
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'grey',
                'fillOpacity': 0.3
            }
        elif prov_affected(prov.decode('utf-8'), included_prov): # has affected vaccine - red
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'red',
                'fillOpacity': 0.3
            }
        else: # has data but no affected vaccine - white
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'white',
                'fillOpacity': 0.3
            }
    return geo_data_copy

In [10]:
m = Map(center=(39.9042, 116.4074), zoom=4)
for vaccination in vaccinations_affected:
    geo_json = GeoJSON(data = prepare_geo_json(vaccination, final_df, geo_data, has_data_provs), name = vaccination)
    m.add_layer(geo_json)
    
m.add_control(LayersControl())
m


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Versions

In [9]:
print ('Versions:\n-------------')
print ("python:")
print (sys.version)
print ('-------------')
print ("pandas:")
pd.show_versions()
print ('-------------')
print ("ipyleaflet:")
print (ipyleaflet.__version__)

Versions:
-------------
python:
3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
-------------
pandas:

INSTALLED VERSIONS
------------------
commit: None
python: 3.6.3.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
byteorder: little
LC_ALL: None
LANG: ZH
LOCALE: None.None

pandas: 0.23.3
pytest: 3.2.1
pip: 9.0.1
setuptools: 40.0.0
Cython: 0.26.1
numpy: 1.14.5
scipy: None
pyarrow: None
xarray: 0.10.8
IPython: 6.4.0
sphinx: 1.6.3
patsy: None
dateutil: 2.7.3
pytz: 2017.2
blosc: None
bottleneck: None
tables: None
numexpr: None
feather: None
matplotlib: None
openpyxl: 2.4.8
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.2
lxml: 4.1.0
bs4: 4.6.0
html5lib: 1.0.1
sqlalchemy: 1.1.13
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None
-------------
ipyleaflet:
0.9.0


GeoJson 来源:

https://github.com/clemsos/d3-china-map/blob/master/maps/zh-mainland-provinces.topo.json

利用 http://jeffpaine.github.io/geojson-topojson/ 将其转化为geojson
